# Module 03: Connect to Azure Cosmos DB SQL API with the SDK

## Import and use the Azure Cosmos DB SQL API SDK

### Understand the SDK

| **Class** | **Description** |
| --- | --- |
| Microsoft.Azure.Cosmos.**CosmosClient** | Client-side logical representation of an Azure Cosmos DB account and the primary class used for the SDK |
| Microsoft.Azure.Cosmos.**Database** | Logically represents a database client-side and includes common operations for database management |
| Microsoft.Azure.Cosmos.**Container** | Logically represents a container client-side and includes common operations for container management |

### Import from package manager

```bash
dotnet add package Microsoft.Azure.Cosmos \
    --version 3.22.1
```

### .NET project file

```xml
<Project Sdk="Microsoft.NET.Sdk">
  <PropertyGroup>
    <OutputType>Exe</OutputType>
    <TargetFramework>net6.0</TargetFramework>
  </PropertyGroup>
  <ItemGroup>
    <PackageReference Include="Microsoft.Azure.Cosmos" Version="3.22.1" />
  </ItemGroup>
</Project>
```

### Connect to an online account - with connection string

In [ ]:
using Microsoft.Azure.Cosmos;

Console.WriteLine($"Using connectionString: {connectionString}");

CosmosClient client = new (connectionString);

client

### Connect to an online account - with endpoint and key

## Configure the Azure Cosmos DB SQL API SDK

In [ ]:
using Microsoft.Azure.Cosmos;

Console.WriteLine($"Using documentEndpoint: {documentEndpoint}");
Console.WriteLine($"Using primaryMasterKey: {primaryMasterKey}");

CosmosClient client = new (documentEndpoint, primaryMasterKey);

client

### Read properties of the account

The **AccountProperties** class includes useful properties such as, but not limited to:

| **Property** | **Description** |
| --- | --- |
| **Id** | Gets the unique name of the account |
| **ReadableRegions** | Gets a list of readable locations for the account |
| **WritableRegions** | Gets a list of writable locations for the account |
| **Consistency** | Gets the default consistency level for the account |

In [ ]:
AccountProperties account = await client.ReadAccountAsync();

account

### Interact with a database

In [ ]:
// Retrieve an existing database
Database database = client.GetDatabase("cosmicworks");

database

In [ ]:
// Create a new database
Database database = await client.CreateDatabaseAsync("cosmicworks");

// Note there can be an exception when the database already exists...
// Error: Microsoft.Azure.Cosmos.CosmosException : Response status code does not indicate success: Conflict (409)

In [ ]:
// Create database if it doesn't already exist
Database database = await client.CreateDatabaseIfNotExistsAsync("cosmicworks");

database

### Interact with a container

In [ ]:
// Retrieve an existing container
Container container = database.GetContainer("products");

container

In [ ]:
// Create a new container
Container container = await database.CreateContainerAsync(
    "cosmicworks", 
    "/categoryId", 
    400 
);

container

In [ ]:
// Create container if it doesn't already exist
Container container = await database.CreateContainerIfNotExistsAsync(
    "cosmicworks", 
    "/categoryId", 
    400 
);

container

## Implement client singleton

Each instance of the CosmosClient class has a few features that are already implemented on your behalf:

- Instances are already thread-safe
- Instances efficiently manage connections


## Configure connectivity mode

![image](https://docs.microsoft.com/en-us/azure/cosmos-db/sql/media/performance-tips/connection-policy.png)

## Gateway mode

- Gateway mode is supported on all SDK platforms. If your application runs within a corporate network with strict firewall restrictions, gateway mode is the best choice because it uses the standard HTTPS port and a single DNS endpoint
- The performance tradeoff, however, is that gateway mode involves an additional network hop every time data is read from or written to Azure Cosmos DB
- We also recommend gateway connection mode when you run applications in environments that have a limited number of socket connections.

## Direct mode

- Direct mode supports connectivity through TCP protocol, using TLS for initial authentication and encrypting traffic, and offers better performance because there are fewer network hops
- The application connects directly to the backend replicas. Direct mode is currently only supported on .NET and Java SDK platforms.

In [ ]:
// Configures the client to use Direct connection mode.
CosmosClientOptions options = new () { ConnectionMode = ConnectionMode.Direct };
CosmosClient client = new (documentEndpoint, primaryMasterKey, options);

client

In [ ]:
// Configures the client to use Gateway connection mode.
CosmosClientOptions options = new () { ConnectionMode = ConnectionMode.Gateway };
CosmosClient client = new (documentEndpoint, primaryMasterKey, options);

client

### Setting the preferred application region[s]

In [ ]:
// Configs single preferred region for client to connect to.
CosmosClientOptions options = new () { ApplicationRegion = Regions.WestEurope };
CosmosClient client = new (documentEndpoint, primaryMasterKey, options);

client

In [ ]:
// Configs the client to use custom failover/priority list.
CosmosClientOptions options = new () { ApplicationPreferredRegions = new List<string> { Regions.WestEurope, Regions.NorthEurope } };
CosmosClient client = new (documentEndpoint, primaryMasterKey, options);

client

## DEMO Tear down

In [ ]:
await database.DeleteAsync();